<a href="https://colab.research.google.com/github/CryptoRobotFr/1-create-personnal-trading-bot/blob/main/trading-bot-live-ftx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Package installation</h1>

In [ ]:
pip install ftx

In [ ]:
pip install ciso8601

In [ ]:
pip install ta

<h1>Import, define Client, dowload data and create indicators</h1>

In [ ]:
import ftx
import pandas as pd
import ta
import time
import json
from math import *

accountName = 'Ytb-Tester'
pairSymbol = 'BTC/USD'
fiatSymbol = 'USD'
cryptoSymbol = 'BTC'
myTruncate = 4

client = ftx.FtxClient(api_key='',
                   api_secret='', subaccount_name=accountName)

data = client.get_historical_data(
    market_name=pairSymbol, 
    resolution=3600, 
    limit=650, 
    start_time=float(
    round(time.time()))-650*3600, 
    end_time=float(round(time.time())))
df = pd.DataFrame(data)

df['SMA200'] = ta.trend.sma_indicator(df['close'], 200)
df['SMA600'] = ta.trend.sma_indicator(df['close'], 600)
print(df)

                     startTime          time  ...     SMA200        SMA600
0    2021-07-07T09:00:00+00:00  1.625648e+12  ...        NaN           NaN
1    2021-07-07T10:00:00+00:00  1.625652e+12  ...        NaN           NaN
2    2021-07-07T11:00:00+00:00  1.625656e+12  ...        NaN           NaN
3    2021-07-07T12:00:00+00:00  1.625659e+12  ...        NaN           NaN
4    2021-07-07T13:00:00+00:00  1.625663e+12  ...        NaN           NaN
..                         ...           ...  ...        ...           ...
645  2021-08-03T06:00:00+00:00  1.627970e+12  ...  39709.650  34873.931667
646  2021-08-03T07:00:00+00:00  1.627974e+12  ...  39725.570  34883.405000
647  2021-08-03T08:00:00+00:00  1.627978e+12  ...  39734.320  34892.923333
648  2021-08-03T09:00:00+00:00  1.627981e+12  ...  39735.975  34902.498333
649  2021-08-03T10:00:00+00:00  1.627985e+12  ...  39739.165  34912.210000

[650 rows x 9 columns]


<h1>Get balance of your account</h1>

In [ ]:
def getBalance(myclient, coin):
    jsonBalance = myclient.get_balances()
    pandaBalance = pd.DataFrame(jsonBalance)
    if pandaBalance.loc[pandaBalance['coin'] == coin].empty : return 0
    else : return float(pandaBalance.loc[pandaBalance['coin'] == coin]['free'])

def truncate(n, decimals=0):
    r = floor(float(n)*10**decimals)/10**decimals
    return str(r)
    

actualPrice = df['close'].iloc[-1]
fiatAmount = getBalance(client, fiatSymbol)
cryptoAmount = getBalance(client, cryptoSymbol)
print(actualPrice, fiatAmount, cryptoAmount)

38715.0 38.58364582 0.0


<h1>Define Buy and Sell condition and place order</h1>

In [ ]:
if float(fiatAmount) > 5 and df['SMA200'].iloc[-2] > df['SMA600'].iloc[-2]:
    quantityBuy = truncate(float(fiatAmount)/actualPrice, myTruncate)
    buyOrder = client.place_order(
        market=pairSymbol, 
        side="buy", 
        price=None, 
        size=quantityBuy, 
        type='market')
    print(buyOrder)

if float(cryptoAmount) > 0.0001 and df['SMA200'].iloc[-2] < df['SMA600'].iloc[-2]:
    buyOrder = client.place_order(
        market=pairSymbol, 
        side="sell", 
        price=None, 
        size=truncate(cryptoAmount, myTruncate), 
        type='market')
    print(buyOrder)

{'id': 68174549224, 'clientId': None, 'market': 'BTC/USD', 'type': 'market', 'side': 'buy', 'price': None, 'size': 0.0009, 'status': 'new', 'filledSize': 0.0, 'remainingSize': 0.0009, 'reduceOnly': False, 'liquidation': None, 'avgFillPrice': None, 'postOnly': False, 'ioc': True, 'createdAt': '2021-08-03T10:51:38.331261+00:00', 'future': None}


<h1>Full code in one cell</h1>

In [ ]:
import ftx
import pandas as pd
import ta
import time
import json
from math import *

accountName = 'Ytb-Tester'
pairSymbol = 'BTC/USD'
fiatSymbol = 'USD'
cryptoSymbol = 'BTC'
myTruncate = 4

client = ftx.FtxClient(api_key='xL6SYTqmHtawoCaihqkoqlCtW_q_vfJ6-xVZlnh6',
                   api_secret='em2tgTy47IblAubmT18_yIAMwzDVXoTnM2hHCvWm', subaccount_name=accountName)

data = client.get_historical_data(
    market_name=pairSymbol, 
    resolution=3600, 
    limit=650, 
    start_time=float(
    round(time.time()))-650*3600, 
    end_time=float(round(time.time())))
df = pd.DataFrame(data)

df['SMA200'] = ta.trend.sma_indicator(df['close'], 200)
df['SMA600'] = ta.trend.sma_indicator(df['close'], 600)
print(df)

def getBalance(myclient, coin):
    jsonBalance = myclient.get_balances()
    pandaBalance = pd.DataFrame(jsonBalance)
    if pandaBalance.loc[pandaBalance['coin'] == coin].empty : return 0
    else : return float(pandaBalance.loc[pandaBalance['coin'] == coin]['free'])

def truncate(n, decimals=0):
    r = floor(float(n)*10**decimals)/10**decimals
    return str(r)
    

actualPrice = df['close'].iloc[-1]
fiatAmount = getBalance(client, fiatSymbol)
cryptoAmount = getBalance(client, cryptoSymbol)
print(actualPrice, fiatAmount, cryptoAmount)

if float(fiatAmount) > 5 and df['SMA200'].iloc[-2] > df['SMA600'].iloc[-2]:
    quantityBuy = truncate(float(fiatAmount)/actualPrice, myTruncate)
    buyOrder = client.place_order(
        market=pairSymbol, 
        side="buy", 
        price=None, 
        size=quantityBuy, 
        type='market')
    print(buyOrder)

if float(cryptoAmount) > 0.0001 and df['SMA200'].iloc[-2] < df['SMA600'].iloc[-2]:
    buyOrder = client.place_order(
        market=pairSymbol, 
        side="sell", 
        price=None, 
        size=truncate(cryptoAmount, myTruncate), 
        type='market')
    print(buyOrder)

                     startTime          time  ...     SMA200        SMA600
0    2021-07-07T09:00:00+00:00  1.625648e+12  ...        NaN           NaN
1    2021-07-07T10:00:00+00:00  1.625652e+12  ...        NaN           NaN
2    2021-07-07T11:00:00+00:00  1.625656e+12  ...        NaN           NaN
3    2021-07-07T12:00:00+00:00  1.625659e+12  ...        NaN           NaN
4    2021-07-07T13:00:00+00:00  1.625663e+12  ...        NaN           NaN
..                         ...           ...  ...        ...           ...
645  2021-08-03T06:00:00+00:00  1.627970e+12  ...  39709.650  34873.931667
646  2021-08-03T07:00:00+00:00  1.627974e+12  ...  39725.570  34883.405000
647  2021-08-03T08:00:00+00:00  1.627978e+12  ...  39734.320  34892.923333
648  2021-08-03T09:00:00+00:00  1.627981e+12  ...  39735.975  34902.498333
649  2021-08-03T10:00:00+00:00  1.627985e+12  ...  39739.075  34912.180000

[650 rows x 9 columns]
38697.0 3.72397408 0.0009
